In [72]:
#import all the necessary modules
import numpy as np # linear algebra
import time
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup
import json

##We will be looking at data from Liverpools 2021/22 season

In [75]:
base_url = 'https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats'

response = requests.get(base_url)

soup = BeautifulSoup(response.content, "html")

print(response)


<Response [200]>


##All of Liverpool's data can be found in the standings table

In [76]:
#The table that we want is under the <table> tab with class = 'stats table'
#lets select the table

standings = soup.select('table', class_ = 'stats_table')


#now lets parse the contents of the table
standings_table = soup.select('table.stats_table')[0]
print(standings_table)

<table class="stats_table sortable min_width force_mobilize" data-cols-to-freeze=",2" id="results2021-202291_overall"> <caption>Regular season Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup> <thead> <tr> <th aria-label="Rank" class="poptip sort_default_asc center" data-stat="rank" data-tip="&lt;strong&gt;Rank&lt;/strong&gt;&lt;br&gt;Squad finish in competition&lt;br&gt;Finish within the league or competition.&lt;br&gt;For knockout competitions may show final round reached.&lt;br&gt;Colors and arrows represent promotion/relegation or qualifiation for continental cups.&lt;br&gt;Trophy indicates team won league whether by playoffs or by leading the table.&lt;br&gt;Star indicates topped table in league USING another means of naming champion." scope="col">Rk</th> <th aria-label="Squad" class="poptip sort_default_asc center" data-stat="team" scope="col">Squad</th> <th aria-label="Matches P

In [77]:
#next we need to find the links for each team, they are under the 'a' class
links = standings_table.find_all('a')
links

#we are only interested in the href links
links_list = []
for l in links:
  links_list.append(l.get('href'))

links_list

['/en/squads/b8fd03ef/2021-2022/Manchester-City-Stats',
 '/en/players/e46012d4/Kevin-De-Bruyne',
 '/en/players/3bb7b8b4/Ederson',
 '/en/squads/822bd0ba/2021-2022/Liverpool-Stats',
 '/en/players/e342ad68/Mohamed-Salah',
 '/en/players/7a2e46a8/Alisson',
 '/en/squads/cff3d9bb/2021-2022/Chelsea-Stats',
 '/en/players/9674002f/Mason-Mount',
 '/en/players/33887998/Edouard-Mendy',
 '/en/squads/361ca564/2021-2022/Tottenham-Hotspur-Stats',
 '/en/players/92e7e919/Son-Heung-min',
 '/en/players/8f62b6ee/Hugo-Lloris',
 '/en/squads/18bb7c10/2021-2022/Arsenal-Stats',
 '/en/players/bc7dc64d/Bukayo-Saka',
 '/en/players/466fb2c5/Aaron-Ramsdale',
 '/en/squads/19538871/2021-2022/Manchester-United-Stats',
 '/en/players/dea698d9/Cristiano-Ronaldo',
 '/en/players/7ba6d84e/David-de-Gea',
 '/en/squads/7c21e445/2021-2022/West-Ham-United-Stats',
 '/en/players/79c84d1c/Jarrod-Bowen',
 '/en/players/9328b835/Lukasz-Fabianski',
 '/en/squads/a2d435b3/2021-2022/Leicester-City-Stats',
 '/en/players/45963054/Jamie-Vardy'

In [78]:
#we only want to look at liverpool
liverpool = 'https://fbref.com' + links_list[3]

liverpool

'https://fbref.com/en/squads/822bd0ba/2021-2022/Liverpool-Stats'

In [79]:
#lets parse the liverpool data
data = requests.get(liverpool)
data.text

'    \n      \n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202301312" />\n<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->\n<script type="text/javascript" async=true>\n    (function() {\n\tvar host = window.location.hostname;\n\tvar element = document.createElement(\'script\');\n\tvar firstScript = document.getElementsByTagName(\'script\')[0];\n\tvar url = \'https://cmp.quantcast.com\'\n\t    .concat(\'/choice/\', \'XwNYEpNeFfhfr\', \'/\', host, \n\t\t    \'/choice.js?tag_version=V2\');\n\tvar uspTries = 0;\n\tvar uspTriesLimit = 3;\n\telement.async = true;\n\telement.type = \'text/javascript\';\n\telement.src = url;\n\t\n\tfirstScript.parentNode.insertBef

In [80]:
#get all scores and fixtures data
#se pandas.read_html function to get the table we want
scores_fixtures = pd.read_html(data.text, match = 'Scores & Fixtures')
scores_fixtures_df = scores_fixtures[0]

scores_fixtures_df = scores_fixtures_df.drop('Match Report', axis=1)
scores_fixtures_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Notes
0,2021-08-14,17:30,Premier League,Matchweek 1,Sat,Away,W,3,0,Norwich City,1.6,1.4,50,27023,James Milner,4-3-3,Andre Marriner,NaN
1,2021-08-21,12:30,Premier League,Matchweek 2,Sat,Home,W,2,0,Burnley,2.7,0.9,65,52591,Jordan Henderson,4-3-3,Mike Dean,NaN
2,2021-08-28,17:30,Premier League,Matchweek 3,Sat,Home,D,1,1,Chelsea,2.5,0.6,65,54000,Jordan Henderson,4-3-3,Anthony Taylor,NaN
3,2021-09-12,16:30,Premier League,Matchweek 4,Sun,Away,W,3,0,Leeds United,4.3,1.2,56,36507,Virgil van Dijk,4-3-3,Craig Pawson,NaN
4,2021-09-15,20:00,Champions Lg,Group stage,Wed,Home,W,3,2,it Milan,2.5,1.6,60,51445,Jordan Henderson,4-3-3,Szymon Marciniak,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2022-05-10,20:00,Premier League,Matchweek 33,Tue,Away,W,2,1,Aston Villa,1.8,1.4,60,41919,Virgil van Dijk,4-3-3,Jonathan Moss,NaN
59,2022-05-14,16:45,FA Cup,Final,Sat,Away,D,0 (6),0 (5),Chelsea,NaN,NaN,53,84897,Jordan Henderson,4-3-3,Craig Pawson,Liverpool won on penalty kicks following extra...
60,2022-05-17,19:45,Premier League,Matchweek 37,Tue,Away,W,2,1,Southampton,1.3,0.1,70,31588,James Milner,4-3-3,Martin Atkinson,NaN
61,2022-05-22,16:00,Premier League,Matchweek 38,Sun,Home,W,3,1,Wolves,2.7,1.3,63,53097,Jordan Henderson,4-3-3,Anthony Taylor,NaN


In [81]:
#im only looking at the scores and fixtures table for Liverpool
soup_liverpool = BeautifulSoup(data.content, 'html')

results = soup_liverpool.select('table', class_ = 'stats_table sortable min_width now_sortable sticky_table eq1 re1 le1 is_sorted', id = 'matchlogs_for')

results = results[1]

#now i want to find all the links of the individual match reports, they can be found in the 'a' tags
reports = results.find_all('a')
reports



[<a href="/en/matches/c52500ad/Norwich-City-Liverpool-August-14-2021-Premier-League">2021-08-14</a>,
 <a href="/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures">Premier League</a>,
 <a href="/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures">Matchweek 1</a>,
 <a href="/en/squads/1c781004/2021-2022/Norwich-City-Stats">Norwich City</a>,
 <a href="/en/players/2f90f6b8/James-Milner">James Milner</a>,
 <a href="/en/matches/c52500ad/Norwich-City-Liverpool-August-14-2021-Premier-League">Match Report</a>,
 <a href="/en/matches/94d9dac0/Liverpool-Burnley-August-21-2021-Premier-League">2021-08-21</a>,
 <a href="/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures">Premier League</a>,
 <a href="/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures">Matchweek 2</a>,
 <a href="/en/squads/943e8050/2021-2022/Burnley-Stats">Burnley</a>,
 <a href="/en/players/935e6b8f/Jordan-Henderson">Jordan Henderson</a>

In [86]:
#get the dates of the matches in the match reports
dates_list = []
for r in reports:
  reports_list.append(r.get('href'))
  r_string = str(r)

  if ('/en/matches/' in r_string) and ('Match Report' not in r_string):
    first_index = r_string.find('>') + 1
    last_index = r_string.rfind('<')
    date_string = r_string[first_index:last_index]
    dates_list.append(date_string)

dates_list
len(dates_list)

63

In [83]:
#get the hrefs of the match reports
reports_list = []
for r in reports:
  reports_list.append(r.get('href'))



#we only want the links with 'matches' in them
match_stats = []
for x in reports_list:
  if '/matches/' in x:
     y =  'https://fbref.com'
     link_string = y + x
     match_stats.append(link_string)



match_stat_list_final = []
for item in match_stats:
  if item not in match_stat_list_final:
    match_stat_list_final.append(item)


match_stat_list_final




['https://fbref.com/en/matches/c52500ad/Norwich-City-Liverpool-August-14-2021-Premier-League',
 'https://fbref.com/en/matches/94d9dac0/Liverpool-Burnley-August-21-2021-Premier-League',
 'https://fbref.com/en/matches/78aa75e6/Liverpool-Chelsea-August-28-2021-Premier-League',
 'https://fbref.com/en/matches/e6a245be/Leeds-United-Liverpool-September-12-2021-Premier-League',
 'https://fbref.com/en/matches/ff3e4ae2/Liverpool-Milan-September-15-2021-Champions-League',
 'https://fbref.com/en/matches/59ef8c18/Liverpool-Crystal-Palace-September-18-2021-Premier-League',
 'https://fbref.com/en/matches/1d32b710/Norwich-City-Liverpool-September-21-2021-EFL-Cup',
 'https://fbref.com/en/matches/87140543/Brentford-Liverpool-September-25-2021-Premier-League',
 'https://fbref.com/en/matches/1c876d93/Porto-Liverpool-September-28-2021-Champions-League',
 'https://fbref.com/en/matches/2598b046/Liverpool-Manchester-City-October-3-2021-Premier-League',
 'https://fbref.com/en/matches/2ce6d340/Watford-Liverpool

In [90]:
#now lets define a function to do this for all matches
#arguments: match_stat_list_final = list of all match stat urls
def get_match_stats(match_stat_list_final):
    
    #define the list to store all the values

    lst = []
    
    with requests.Session() as session:
        #lets try getting the first batch
        for i in range(0,len(match_stat_list_final+1)):
            match_url = match_stat_list_final[i]
            request = session.get(match_url)
            if request.status_code == 429:
                # Wait for a certain amount of time (e.g., 5 seconds) and then retry
                time.sleep(10)
                request = requests.get(match_url)
            #find the index of the last instance of the character '/'
            #we want to check if liverpool was the home team etc. '/Liverpool' or away
            last = match_stat_list_final[i].rfind('/')

            if match_stat_list_final[i][last+1:last+10] == 'Liverpool':
                Venue = 'Home'

            else:
                Venue = 'Away'

            #get the date of the match
            date = dates_list[i]


            match_stat_table = pd.read_html(request.text, match = 'Liverpool Player Stats')
            #EXTRACT the last row of the df, we only want the team's stats for that match
            #we are also just looking at the summary table, particularly the last row that contains the combined team's stats

            #EXTRACT the last row of the df, we only want the team's stats for that match
            #match[0] : gives match summary
            #match[1] : passing summary
            #match[2] : pass types
            #match[3] : defensive actions
            #match[4] : possession
            #match[5] : misc stats
            final = match_stat_table[0].iloc[[-1]] 
        
            try:
                performance = final[['Performance'][0]]
                expected = final[['Expected'][0]]
                sca = final[['SCA'][0]]
                passes = final[['Passes'][0]]
                #rename variable 'Att' so no key clash
                passes = passes.rename({'Att':'Pass_att'}, axis=1)
                carries = final[['Carries'][0]]
                take_ons = final[['Take-Ons'][0]]
                #rename column 'Att' so the meaning is clearer
                take_ons = take_ons.rename({'Att':'Take_on_att'}, axis=1)
            
            except KeyError or ValueError:
                continue


            #concat all the df
            try:
                final_values1 = pd.concat([performance,expected], axis=1)
                final_values2 = pd.concat([final_values1,sca,passes,carries,take_ons], axis=1)
                #get the row values only
                #get the row values only
                values = final_values2.iloc[0].tolist()
                #append the addtional value 'Venue'
                values.append(Venue)

                #append the addtional value 'Date'
                values.append(date)
                
                lst.append(values)


            except ValueError:
                continue

          
    #lastly, create the dataframe
    df_final = pd.DataFrame(data = lst, columns = ['Goals','Assists','PK','PKatt','Sh','SoT','CrdY','CrdR','Touches','Tkl','Int','Blocks','xG','npxG','xAG'
            'SCA','GCA','Cmp','Cmp_att','Cmp%','PrgP','Carries','PrgC','Tackle_att','Tackle_succ','Venue','Venue_type','Match_date'])
    

    return df_final



In [91]:
#run the function

df_match_stats = get_match_stats(match_stat_list_final)



In [92]:
df_match_stats

,Goals,Assists,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Tkl,...,Cmp_att,Cmp%,PrgP,Carries,PrgC,Tackle_att,Tackle_succ,Venue,Venue_type,Match_date
0,3.0,2.0,0.0,0.0,19.0,6.0,1.0,0.0,666.0,16.0,...,459.0,560.0,82.0,47.0,392.0,25.0,20.0,7.0,Away,2021-08-14
1,2.0,2.0,0.0,0.0,27.0,8.0,0.0,0.0,702.0,14.0,...,471.0,585.0,80.5,51.0,415.0,26.0,12.0,4.0,Home,2021-08-21
2,1.0,0.0,1.0,1.0,23.0,6.0,0.0,0.0,875.0,16.0,...,631.0,758.0,83.2,70.0,442.0,25.0,12.0,4.0,Home,2021-08-28
3,3.0,2.0,0.0,0.0,30.0,8.0,1.0,0.0,684.0,8.0,...,478.0,575.0,83.1,47.0,480.0,29.0,18.0,9.0,Away,2021-09-12
4,2.0,1.0,0.0,1.0,22.0,7.0,1.0,0.0,743.0,24.0,...,502.0,602.0,83.4,56.0,494.0,30.0,41.0,18.0,Home,2021-09-15
5,3.0,1.0,0.0,0.0,25.0,10.0,3.0,0.0,682.0,21.0,...,443.0,549.0,80.7,54.0,386.0,16.0,21.0,12.0,Home,2021-09-18
6,3.0,3.0,0.0,0.0,16.0,5.0,1.0,0.0,702.0,11.0,...,493.0,611.0,80.7,45.0,415.0,23.0,15.0,5.0,Away,2021-09-25
7,5.0,3.0,0.0,0.0,21.0,13.0,0.0,0.0,807.0,18.0,...,583.0,692.0,84.2,56.0,399.0,27.0,22.0,14.0,Away,2021-09-28
8,2.0,2.0,0.0,0.0,6.0,3.0,3.0,0.0,655.0,17.0,...,457.0,554.0,82.5,34.0,382.0,15.0,14.0,10.0,Home,2021-10-03


In [93]:
#rename columns so no clash
df_match_stats = df_match_stats.rename({'xAGSCA':'xAG', 'GCA':'SCA', 'Cmp':'GCA', 'Cmp_att':'Cmp', 'Cmp%':'Cmp_att', 'PrgP':'Cmp%', 'Carries':'PrgP', 'PrgC':'Carries', 'Tackle_att':'PrgC', 'Tackle_succ':'Tackle_att', 'Venue':'Tackle_succ','Venue_type':'Venue'}, axis=1)

df_match_stats

,Goals,Assists,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Tkl,...,Cmp,Cmp_att,Cmp%,PrgP,Carries,PrgC,Tackle_att,Tackle_succ,Venue,Match_date
0,3.0,2.0,0.0,0.0,19.0,6.0,1.0,0.0,666.0,16.0,...,459.0,560.0,82.0,47.0,392.0,25.0,20.0,7.0,Away,2021-08-14
1,2.0,2.0,0.0,0.0,27.0,8.0,0.0,0.0,702.0,14.0,...,471.0,585.0,80.5,51.0,415.0,26.0,12.0,4.0,Home,2021-08-21
2,1.0,0.0,1.0,1.0,23.0,6.0,0.0,0.0,875.0,16.0,...,631.0,758.0,83.2,70.0,442.0,25.0,12.0,4.0,Home,2021-08-28
3,3.0,2.0,0.0,0.0,30.0,8.0,1.0,0.0,684.0,8.0,...,478.0,575.0,83.1,47.0,480.0,29.0,18.0,9.0,Away,2021-09-12
4,2.0,1.0,0.0,1.0,22.0,7.0,1.0,0.0,743.0,24.0,...,502.0,602.0,83.4,56.0,494.0,30.0,41.0,18.0,Home,2021-09-15
5,3.0,1.0,0.0,0.0,25.0,10.0,3.0,0.0,682.0,21.0,...,443.0,549.0,80.7,54.0,386.0,16.0,21.0,12.0,Home,2021-09-18
6,3.0,3.0,0.0,0.0,16.0,5.0,1.0,0.0,702.0,11.0,...,493.0,611.0,80.7,45.0,415.0,23.0,15.0,5.0,Away,2021-09-25
7,5.0,3.0,0.0,0.0,21.0,13.0,0.0,0.0,807.0,18.0,...,583.0,692.0,84.2,56.0,399.0,27.0,22.0,14.0,Away,2021-09-28
8,2.0,2.0,0.0,0.0,6.0,3.0,3.0,0.0,655.0,17.0,...,457.0,554.0,82.5,34.0,382.0,15.0,14.0,10.0,Home,2021-10-03


In [95]:
#join the dataframe with the scores & fixtures dataframe
df_match_stats = df_match_stats.merge(scores_fixtures_df, left_on = ['Venue','Match_date'], right_on = ['Venue','Date'], how = 'left')

df_match_stats

,Goals,Assists,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Tkl,...,GA_y,Opponent_y,xG,xGA_y,Poss_y,Attendance_y,Captain_y,Formation_y,Referee_y,Notes_y
0,3.0,2.0,0.0,0.0,19.0,6.0,1.0,0.0,666.0,16.0,...,0,Norwich City,1.6,1.4,50,27023,James Milner,4-3-3,Andre Marriner,NaN
1,2.0,2.0,0.0,0.0,27.0,8.0,0.0,0.0,702.0,14.0,...,0,Burnley,2.7,0.9,65,52591,Jordan Henderson,4-3-3,Mike Dean,NaN
2,1.0,0.0,1.0,1.0,23.0,6.0,0.0,0.0,875.0,16.0,...,1,Chelsea,2.5,0.6,65,54000,Jordan Henderson,4-3-3,Anthony Taylor,NaN
3,3.0,2.0,0.0,0.0,30.0,8.0,1.0,0.0,684.0,8.0,...,0,Leeds United,4.3,1.2,56,36507,Virgil van Dijk,4-3-3,Craig Pawson,NaN
4,2.0,1.0,0.0,1.0,22.0,7.0,1.0,0.0,743.0,24.0,...,2,it Milan,2.5,1.6,60,51445,Jordan Henderson,4-3-3,Szymon Marciniak,NaN
5,3.0,1.0,0.0,0.0,25.0,10.0,3.0,0.0,682.0,21.0,...,0,Crystal Palace,3.1,0.6,59,52000,Jordan Henderson,4-3-3,Andy Madley,NaN
6,3.0,3.0,0.0,0.0,16.0,5.0,1.0,0.0,702.0,11.0,...,3,Brentford,2.8,2.1,67,16876,Jordan Henderson,4-3-3,Stuart Attwell,NaN
7,5.0,3.0,0.0,0.0,21.0,13.0,0.0,0.0,807.0,18.0,...,1,pt Porto,3.4,0.9,66,23520,Jordan Henderson,4-3-3,Sergey Karasev,NaN
8,2.0,2.0,0.0,0.0,6.0,3.0,3.0,0.0,655.0,17.0,...,2,Manchester City,0.9,1.2,49,53102,Jordan Henderson,4-3-3,Paul Tierney,NaN


In [96]:
#save the dataframe into the path as a csv
outname = 'liverpool.csv'

outdir = 'C:/Users/Ryan Goh/Desktop/liverpool_data/data/raw'
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
df_match_stats.to_csv(f"{outdir}/{outname}")